## 00 packages

In [1]:
##########################################################################################
##########################################################################################

import networkx as nx

from shapely.geometry import Point,Polygon

import math

import random

##########################################################################################
##########################################################################################

import numpy as np

import pandas as pd

##########################################################################################
##########################################################################################

import matplotlib

import matplotlib.pyplot as plt

from matplotlib.ticker import MultipleLocator, FormatStrFormatter

##########################################################################################
##########################################################################################

from itertools import combinations, permutations

from itertools import islice

##########################################################################################
##########################################################################################

import copy


### 01 parameters

In [2]:
##########################################################################################
##########################################################################################

alpha=1

batch_interval=1

##########################################################################################
##########################################################################################

speed=10000/60

##########################################################################################
##########################################################################################

speed

166.66666666666666

### 01 Comparision of the latest compeletion time

In [3]:
##########################################################################################
##########################################################################################

lambda_values = [5,10,20]

patterns=['uniform','centralized']#'centralized','uniform'

##########################################################################################
##########################################################################################

save_paths=["./03result/01Direct_Delivery/","./03result/02Dynamic_VRPPD/","./03result/03Multi-hop/"]

cup_times=[0.00,0.12,0.01,0.00,0.25,0.03,0.00,0.77,0.08,0.00,0.13,0.01,0.00,0.29,0.03,0.00,0.69,0.07]

##########################################################################################
##########################################################################################

def calculate_distance(schedules):
    
    tt_dis=0
    
    for schedule in schedules.values():
        
        trajectory=list(schedule.values())

        for i in range(1, len(trajectory),1):
                
            tt_dis+=Point(trajectory[i-1]).distance(Point(trajectory[i]))
                
    return tt_dis/1000.0

##########################################################################################
##########################################################################################

def flatten_list(lst):
    flattened = []
    for item in lst:
        if isinstance(item, list):
            flattened.extend(flatten_list(item))
        else:
            flattened.append(item)
    return flattened

##########################################################################################
##########################################################################################

arrays=list()

##########################################################################################
##########################################################################################


k=0

for pattern in patterns:

    for lambda_value in lambda_values:
        
        ##########################################################################################
        ##########################################################################################
        
        instance_name=pattern+'_'+str(lambda_value)
        
        ##########################################################################################
        ##########################################################################################
        
        for i in range(3):
            
            ##########################################################################################
            ##########################################################################################
            
            save_path=save_paths[i]
            
            save_path=save_path+pattern+'_'+str(lambda_value)
            
            ##########################################################################################
            ##########################################################################################
            
            parcel_df=pd.read_csv(save_path+"/parcel_df_"+str(alpha)+"_"+str(batch_interval)+".csv")
            
            parcel_df=parcel_df.drop(columns=['Unnamed: 0'])
            
            ##########################################################################################
            ##########################################################################################
            
            if i==0:
                
                based_df=copy.copy(parcel_df[['parcel_id','t_a','pick_up','h1']])
                
            elif i==2:
                
                parcel_df=parcel_df[['parcel_id', 'drop_off','h2','t_dl', 'courier_id','drop_off_time']]

                parcel_df=parcel_df.merge(based_df,on='parcel_id')
            
            ##########################################################################################
            ##########################################################################################
            
            parcel_df['pick_up']=parcel_df.apply(lambda x:tuple([round(float(y),2) for y in x.pick_up.replace('(','').replace(')','').split(',')]),axis=1)
            
            parcel_df['drop_off']=parcel_df.apply(lambda x:tuple([round(float(y),2) for y in x.drop_off.replace('(','').replace(')','').split(',')]),axis=1)
            
            parcel_df['shortest_distance']=parcel_df.apply(lambda x:Point(x.pick_up).distance(Point(x.drop_off)), axis=1)
            
            if i==0:
                
                trajectory_schedules=np.load(save_path+"/courier_schedules_"+str(alpha)+"_"+str(batch_interval)+'.npy',allow_pickle=True).item()
                
            else:
            
                trajectory_schedules=np.load(save_path+"/trajectory_schedules_"+str(alpha)+"_"+str(batch_interval)+'.npy',allow_pickle=True).item()
                
                courier_schedules=np.load(save_path+"/courier_schedules_"+str(alpha)+"_"+str(batch_interval)+'.npy',allow_pickle=True).item()
            
            ##########################################################################################
            ##########################################################################################
            
            # 01 Total travel distance
            
            total_travel_distance=calculate_distance(trajectory_schedules)
            
            ##########################################################################################
            ##########################################################################################
            
            # 02 Average lateness
            
            parcel_df['lateness']=parcel_df.apply(lambda x:x.drop_off_time-x.t_dl if x.drop_off_time>x.t_dl else 0, axis=1)
            
            average_lateness=parcel_df.lateness.mean()
            
            ##########################################################################################
            ##########################################################################################
            
            #03 Average delivery distance
            
            if i==0:
            
                average_delivery_distance=parcel_df.shortest_distance.mean()/1000.0
                
            else:
                
                parcel_schedules=np.load(save_path+"/parcel_schedules_"+str(alpha)+"_"+str(batch_interval)+'.npy',allow_pickle=True).item()
                
                delivery_distances=list()
                
                for schedule in parcel_schedules.values():
                    
                    actual_delivery_time=0
                    
                    time_arrays=flatten_list([[y[1] for y in list(x.values())] for x in schedule.values()])
                    
                    for time_line in schedule.values():
                    
                        actual_delivery_time+=time_line['drop_off'][1]-time_line['pick_up'][1]
                        
                    actual_delivery_distance=actual_delivery_time*speed/1000.0
                    
                    delivery_distances.append(actual_delivery_distance)
                    
                average_delivery_distance=np.array(delivery_distances).mean()
                    
            average_delivery_distance=round(average_delivery_distance,0)
            
            ##########################################################################################
            ##########################################################################################
            
            #04 Average detour ratio
            
            average_detour_ratio=average_delivery_distance/(parcel_df.shortest_distance.mean()/1000.0)
            
            ##########################################################################################
            ##########################################################################################

            # 05 Average request-to-pickup time
            
            if i!=2:
            
                parcel_df['request_to_pickup']=parcel_df.apply(lambda x:x.pick_up_time-x.t_a, axis=1)

                average_request_to_pickup=parcel_df.request_to_pickup.mean()
                
            else:
                
                request_to_pickups=list()
                
                for parcel_id,schedule in parcel_schedules.items():
                    
                    time_arrays=flatten_list([[y[1] for y in list(x.values())] for x in schedule.values()])
                    
                    request_to_pickup=float(time_arrays[0]-float(parcel_df.loc[parcel_df.parcel_id==parcel_id,'t_a']))
                    
                    request_to_pickups.append(request_to_pickup)
                    
                average_request_to_pickup=np.array(request_to_pickups).mean()
                    
            
            ##########################################################################################
            ##########################################################################################

            # 06 Average delivery time
            
            if i!=2:
                
                parcel_df['delivery_time']=parcel_df.apply(lambda x:x.drop_off_time-x.pick_up_time, axis=1)
            
                average_delivery_time=parcel_df.delivery_time.mean()
                
            else:
                
                delivery_times=list()
                
                for parcel_id,schedule in parcel_schedules.items():
                    
                    time_arrays=flatten_list([[y[1] for y in list(x.values())] for x in schedule.values()])
                    
                    delivery_time=time_arrays[-1]-time_arrays[0]
                    
                    delivery_times.append(delivery_time)
                    
                average_delivery_time=np.array(delivery_times).mean()
                
            ##########################################################################################
            ##########################################################################################

            # 06 Average request-to-completion time
            
            average_request_to_completion_time=average_delivery_time+average_request_to_pickup

    
            ##########################################################################################
            ##########################################################################################

            # 07 Average CPU time
            
            average_cpu_time=cup_times[k]
            
            k+=1
            
            ##########################################################################################
            ##########################################################################################
            
            arrays.append(total_travel_distance)
            arrays.append(average_lateness)
            arrays.append(average_delivery_distance) 
            arrays.append(average_detour_ratio)
            arrays.append(average_request_to_pickup)
            arrays.append(average_delivery_time)
            arrays.append(average_request_to_completion_time)
            arrays.append(average_cpu_time)

                
print(arrays)



[6183.002035654076, 402.966, 4.0, 0.9920238530120558, 558.472, 26.586, 585.058, 0.0, 1908.0006530599287, 19.373, 13.0, 3.224077522289181, 83.376, 80.394, 163.77, 0.12, 1524.0003182580385, 4.673, 14.0, 3.472083485542195, 17.81, 112.192, 130.00199999999998, 0.01, 6089.001773484543, 434.276, 4.0, 1.00727639863394, 589.098, 26.196, 615.294, 0.0, 1915.0006820420133, 48.508, 14.0, 3.5254673952187896, 128.653, 82.314, 210.96699999999998, 0.25, 1131.0001959011427, 2.955, 13.0, 3.273648295560305, 18.446, 102.862, 121.30799999999999, 0.03, 6197.001755957013, 472.25, 4.0, 0.9819224184610289, 632.999, 26.856, 659.855, 0.0, 1875.0003862127294, 60.227, 14.0, 3.436728464613601, 147.053, 82.764, 229.817, 0.77, 973.0002970237153, 2.186, 13.0, 3.191247859998344, 19.65, 101.204, 120.85399999999998, 0.08, 5947.001845764247, 413.655, 3.0, 0.9741494589644025, 534.797, 20.304, 555.101, 0.0, 1177.001160070896, 5.681, 12.0, 3.9051234046509893, 13.764, 74.358, 88.122, 0.13, 880.0007175114396, 8.787, 14.0, 4.651

In [4]:

result='''
\\begin{landscape}
\\thispagestyle{empty}
\\renewcommand\\arraystretch{1.2}
\\begin{table}[]
\\centering
\\small
\\caption{Comparisons among the proposed MDSDL and other two benchmarks without spatial division of labor in six instances. Each instance is generated 10 times to report the average result. The name of
each instance consists of the distribution pattern of parcel requests and the value of $\lambda$.}
\\begin{tabular}{p{2.5cm}|p{2.5cm}|cccccccc}
\\toprule
\\multirow{2}{*}{Instance}&\multirow{2}{*}{Methods}& Total &Average& Average&Average&Average&Average&Average&Average\\
&&&travel distance [km] & lateness [min] & delivery distance [km] & detour ratio& RtP time [min] & delivery time [min]& RtC time [min]& CPU time [min]\\\\\\midrule
\\multirow{3}{*}{uniform 5}&\\textit{Direct Deliveries}& %.1f&%.1f & %.1f & %.1f  & %.1f & %.1f& %.1f& %.2f\\\\
&\\textit{Dynamic VRPPD}& %.1f&%.1f & %.1f & %.1f  & %.1f & %.1f& %.1f& %.2f\\\\
&\\textit {MDSDL}& %.1f&%.1f & %.1f & %.1f  & %.1f & %.1f& %.1f& %.2f\\\\\\midrule
\\multirow{3}{*}{uniform 10}&\\textit{Direct Deliveries}& %.1f&%.1f & %.1f & %.1f  & %.1f & %.1f& %.1f& %.2f\\\\
&\\textit{Dynamic VRPPD}& %.1f&%.1f & %.1f & %.1f  & %.1f & %.1f& %.1f& %.2f\\\\
&\\textit {MDSDL}& %.1f&%.1f & %.1f & %.1f  & %.1f & %.1f& %.1f& %.2f\\\\\\midrule
\\multirow{3}{*}{uniform 20}&\\textit{Direct Deliveries}& %.1f&%.1f & %.1f & %.1f  & %.1f & %.1f& %.1f& %.2f\\\\
&\\textit{Dynamic VRPPD}& %.1f&%.1f & %.1f & %.1f  & %.1f & %.1f& %.1f& %.2f\\\\
&\\textit {MDSDL}& %.1f&%.1f & %.1f & %.1f  & %.1f & %.1f& %.1f& %.2f\\\\\\midrule
\\multirow{3}{*}{mono-centric 5}&\\textit{Direct Deliveries}& %.1f&%.1f & %.1f & %.1f  & %.1f & %.1f& %.1f& %.2f\\\\
&\\textit{Dynamic VRPPD}& %.1f&%.1f & %.1f & %.1f  & %.1f & %.1f& %.1f& %.2f\\\\
&\\textit {MDSDL}& %.1f&%.1f & %.1f & %.1f  & %.1f & %.1f& %.1f& %.2f\\\\\\midrule
\\multirow{3}{*}{mono-centric 10}&\\textit{Direct Deliveries}& %.1f&%.1f & %.1f & %.1f  & %.1f & %.1f& %.1f& %.2f\\\\
&\\textit{Dynamic VRPPD}& %.1f&%.1f & %.1f & %.1f  & %.1f & %.1f& %.1f& %.2f\\\\
&\\textit {MDSDL}& %.1f&%.1f & %.1f & %.1f  & %.1f & %.1f& %.1f& %.2f\\\\\\midrule
\\multirow{3}{*}{mono-centric 20}&\\textit{Direct Deliveries}& %.1f&%.1f & %.1f & %.1f  & %.1f & %.1f& %.1f& %.2f\\\\
&\\textit{Dynamic VRPPD}& %.1f&%.1f & %.1f & %.1f  & %.1f & %.1f& %.1f& %.2f\\\\
&\\textit {MDSDL}& %.1f&%.1f & %.1f & %.1f  & %.1f & %.1f& %.1f& %.2f\\\\
\\bottomrule
\\end{tabular}
\\label{Table:Results}
\\end{table}
\\vfill
\\end{landscape}
'''%tuple(arrays)

result=result.replace("\n ","\n")

print(result.replace("%",""))


\begin{landscape}
\thispagestyle{empty}
\renewcommand\arraystretch{1.2}
\begin{table}[]
\centering
\small
\caption{Comparisons among the proposed MDSDL and other two benchmarks without spatial division of labor in six instances. Each instance is generated 10 times to report the average result. The name of
each instance consists of the distribution pattern of parcel requests and the value of $\lambda$.}
\begin{tabular}{p{2.5cm}|p{2.5cm}|cccccccc}
\toprule
\multirow{2}{*}{Instance}&\multirow{2}{*}{Methods}& Total &Average& Average&Average&Average&Average&Average&Average\
&&&travel distance [km] & lateness [min] & delivery distance [km] & detour ratio& RtP time [min] & delivery time [min]& RtC time [min]& CPU time [min]\\\midrule
\multirow{3}{*}{uniform 5}&\textit{Direct Deliveries}& 6183.0&403.0 & 4.0 & 1.0  & 558.5 & 26.6& 585.1& 0.00\\
&\textit{Dynamic VRPPD}& 1908.0&19.4 & 13.0 & 3.2  & 83.4 & 80.4& 163.8& 0.12\\
&\textit {MDSDL}& 1524.0&4.7 & 14.0 & 3.5  & 17.8 & 112.2& 130.0& 0.01\

In [5]:
parcel_schedules['p7']

{'c2': {'pick_up': ((-1000.0, 0.0), 7), 'drop_off': ((500.0, 866.03), 61)},
 'c9': {'pick_up': ((500.0, 866.03), 63), 'drop_off': ((2500.0, 866.03), 87)},
 'c17': {'pick_up': ((2500.0, 866.03), 105),
  'drop_off': ((3000.0, 1732.05), 123)}}

In [6]:
courier_schedules['c17'][57]

((4000.0, 1732.05), ['park'])

In [7]:
parcel_df.loc[parcel_df.parcel_id=='p7']

,parcel_id,drop_off,h2,t_dl,courier_id,drop_off_time,t_a,pick_up,h1,shortest_distance,lateness
6,p7,"(3000.0, 1732.05)",H8,216,c17,123.0,0,"(500.0, -866.03)",H0,3605.554005,0.0
